In [ ]:
# [ 결정 트리 ]
# : 예/아니오 질문으로 학습 진행
# - 스무고개 : 질문이 가장 중요하다
# - 주요 피처 찾는 것도 가능 => 전처리 대신 해준다?!?!

# [ 동작 알고리즘 ]
# : 어떤 피처로 질문하는지에 따라 성능 좌우 
# => 분할 속성 : 
# 
# [ 불순도 ]
# : 서로 다른 데이터가 섞인 정도
# - 정보 이득 : 분할 후 불순도 차이
# - 수치화 방법
# [ 1. Entropy ]
# : 정보 이득 측정값으로 불순도 수치화 => 0에 가깝도록
#   알아서 정보이득이 좋은 분할 속성을 선택
# - 단점 : 혼자 있으면 약하다

# [ 2. Gini 계수 ]
# : 균일도로 불순도 수치화
# 우린 엔트로피 혹은 지니 계수 중에서 고르면 됨


# [ Parameters of Decision Tree ]
# 1) criterion='gini'   : 분할 품질 측정 ('log_loss', 'gini', 'entropy')
# 2) x_depth=None       : Tree 최대 깊이; 클수록 모델 복잡도 증가
# 3) min_sample_split=2 : 
# 4) spliter='best'     : or random (tree가 정말 많으면 상관 없/ 속도 빨라짐)

# 성능이 좋지 않다;; 앙상블로 가는 길

<hr>

1. iris 에서 Decision Tree로 그래프 그리기
2. HousingData로 최고의 피처 찾기

1. Iris.csv

In [42]:
# 1. Load Data
import pandas as pd
irisDF = pd.read_csv("../../DATA/iris.csv")
# irisDF.info()

# 2. Split
# 1) target, feature
target = irisDF.species
feature = irisDF[['sepal_length', 'sepal_width']]
print('Check feature & target :', feature.shape, target.shape)

# 2) train, val, test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, 
                                                    random_state=11, stratify=target)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2,
                                                  random_state=11, stratify=y_train)

# 3. Scaling
# 1) check standardized properties
# print('train set :', X_train.mean(), X_train.std())
# print('val set : ', X_val.mean(), X_val.std())
# print('test set : ', X_test.mean(), X_test.std())
# 엥 안된다,, 시리즈로 출력됨

# 2) Standard Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# - print shape
print('X_train_scaled : ', X_train_scaled.shape)
print('X_val_scaled : ', X_val_scaled.shape)
print('X_test_scaled : ', X_test_scaled.shape)

Check feature & target : (150, 2) (150,)
X_train_scaled :  (96, 2)
X_val_scaled :  (24, 2)
X_test_scaled :  (30, 2)


In [47]:
# 2. Set model : SVM
from sklearn.svm import SVC
model = SVC(kernel='linear', random_state=11)
model.fit(X_train_scaled, y_train)

# 3. Draw plot
import matplotlib.pyplot as plt
import numpy as np

# 1) Switch y_test classes into numbers
y_test_num = y_test.map({'setosa': 0, 'versicolor': 1, 'virginica': 2})

def plot_decision_regions(model, x_data, y_data):
    plt.figure(figsize=(6, 6))
    # 결정경계선 그리기
    a = model.coef_.T[0]    # 계수1
    b = model.coef_.T[1]    # 계수2
    c = model.intercept_    # 절편
    # 식 : ax+by+c = 0 --> y = -a/b * x_data - c/b
    y = -a/b*x_data - c/b
    margin = 1 / np.sqrt(a**2 + b**2)   # margin 값 연산


    # Setosa
    plt.scatter(x_data[:50],y_data[:50], label = 'Setosa')
    # Versicolor
    plt.scatter(x_data[50:],y_data[50:], label = 'Versicolor')
    plt.legend()
    # 결정 경계선
    xx = [min(x_data), max(x_data)]
    yy = (-a/b*xx - c/b)
    plt.plot(xx,yy, 'r-')
    # 결정 경계선 위 아래
    yy_down = yy - np.sqrt(1+a**2)*margin
    yy_up = yy + np.sqrt(1+a**2)*margin
    plt.plot(xx,yy_down,'k--')
    plt.plot(xx,yy_up,'k--')
    # 서포트 벡터
    sv_x = model.support_vectors_.T[0]
    sv_y = model.support_vectors_.T[1]
    plt.plot(sv_x, sv_y, 'ko', alpha = 0.8,label = 'Support vector')
    plt.title('[decision boundary]')


plot_decision_regions(model, feature.iloc[:, 0], feature.iloc[:, 1])

ValueError: operands could not be broadcast together with shapes (3,) (150,) 

<Figure size 600x600 with 0 Axes>

<hr>

HousingData.csv

In [23]:
# 1. Load Data
import pandas as pd

HouseDF = pd.read_csv('../../DATA/HousingData.csv')
HouseDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     486 non-null    float64
 1   ZN       486 non-null    float64
 2   INDUS    486 non-null    float64
 3   CHAS     486 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      486 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    486 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [51]:

# 2. Split Dataset
# 1) f & t
feature = HouseDF.drop('MEDV', axis=1)
target = HouseDF.MEDV
print('Shapes :', feature.shape, target.shape)

# 2) Fill NaN with median
feature.fillna(feature.median(), inplace=True)

# 3) tr & te
X_train, X_test, y_train, y_test = train_test_split(feature, target, train_size=0.8, random_state=11)

# 3. Set model : Decision Tree Classifier
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state=11)
dt.fit(X_train, y_train)

# 1) Check performance : r2_score, cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


y_pred = dt.predict(X_test)
print('R2 score :', round(r2_score(y_test, y_pred), 4))
print('Cross Validation Score :', cross_val_score(dt, feature, target, cv=5))
print()

# 2) 주성분 분석
print('Feature importance :', dt.feature_importances_)

Shapes : (506, 13) (506,)
R2 score : 0.7182
Cross Validation Score : [ 0.49478933  0.36316549  0.58792072  0.13011857 -1.14556382]

Feature importance : [3.16633862e-02 1.03263531e-03 2.40557696e-03 9.48038311e-03
 5.59199278e-02 5.81336317e-01 8.86657552e-03 4.16166952e-02
 4.28184872e-04 1.08786429e-02 8.70091945e-03 8.76764113e-03
 2.38903114e-01]
